pip install textgrad

In [6]:
import textgrad as tg

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify the API key is loaded
if os.getenv("OPENAI_API_KEY") is None:
    raise ValueError("OPENAI_API_KEY not found in environment variables")


tg.set_backward_engine("gpt-4o", override=True)

# Step 1: Get an initial response from an LLM
model = tg.BlackboxLLM("gpt-4o")
question_string = ("If it takes 1 hour to dry 25 shirts under the sun, "
                    "how long will it take to dry 30 shirts under the sun? "
                    "Reason step by step.")

question = tg.Variable(question_string, role_description="question to the LLM", requires_grad=False)

# Step 2: Get the LLM's response
answer = model(question)
print(answer)

To determine how long it will take to dry 30 shirts under the sun, we need to consider the drying process and whether it is affected by the number of shirts.

1. **Understand the Drying Process**: Drying shirts under the sun is typically a parallel process. Each shirt dries independently of the others, assuming there is enough space and sunlight for all shirts to be exposed equally.

2. **Initial Information**: We know that 25 shirts take 1 hour to dry. This implies that each shirt, when exposed to the sun, takes 1 hour to dry.

3. **Drying 30 Shirts**: Since drying is a parallel process and each shirt dries independently, adding more shirts does not increase the drying time for each shirt. Therefore, drying 30 shirts will also take 1 hour, provided that all shirts have equal exposure to sunlight and there is no limitation in space or sunlight.

4. **Conclusion**: The time it takes to dry 30 shirts is the same as the time it takes to dry 25 shirts, which is 1 hour, assuming all conditi

In [7]:
answer.set_role_description("concise and accurate answer to the question")

optimizer = tg.TGD(parameters=[answer], verbose=1)

evaluation_instruction = (f"Here's a question: {question_string}. "
                           "Evaluate any given answer to this question, "
                           "be smart, logical, and very critical. "
                           "Just provide concise feedback.")

loss_fn = tg.TextLoss(evaluation_instruction)


In [ ]:
loss = loss_fn(answer)
loss.backward()
optimizer.step()
answer